<a href="https://colab.research.google.com/github/Zbrlaa/RI_TP1/blob/main/1-Recherche%20d'information%20classique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Partie 1. - Recherche d'Information classique

Dans cette partie, nous allons mettre en œuvre des principes et modèles classiques de Recherche d'Information. Le jeu de données est une collection de livres au format texte (.txt) de Henry Rider Haggard. Jetez un œil à ces documents dans le dossier _data_.

En sortie de ce module, vous serez capable de :

- Construire un index inversé ;
- Effectuer des requêtes simples selon le modèle booléen :
- Calculer la pondération des termes selon la méthode TF-IDF ;
- Mettre en œuvre le modèle vectoriel de recherche d'information et y appliquer des requêtes.

### Import des bibliothèques logicielles et configuration

Les lignes suivantes permettent d'instancier un objet de la classe `IRSystem` représentant notre moteur de recherche et de charger les données en RAM.

In [4]:
import os

# Vérifie si le code est exécuté sur Google Colab
if 'COLAB_GPU' in os.environ:
    # Commandes à exécuter uniquement sur Google Colab
    !git clone https://github.com/vincentmartin/tp-information-retrieval-with-llm-student-version.git
    %cd tp-information-retrieval-with-llm-student-version
else:
    # Commandes à exécuter si ce n'est pas sur Google Colab
    print("Pas sur Google Colab, ces commandes ne seront pas exécutées.")

Pas sur Google Colab, ces commandes ne seront pas exécutées.


#### Chargement des données

Les lignes ci-dessous permettent de charger les données qui sont un ensemble de 60 livres au format texte (.txt) d'[Henry Rider Haggard](https://fr.wikipedia.org/wiki/Henry_Rider_Haggard).


In [3]:
from classic_ir.IRSystem import *

# !rm -rf ./data/RiderHaggard/stemmed
ir_system = IRSystem()
ir_system.read_data('./data/RiderHaggard') # chargement des données et prétraitement des documents (stemming).

/home/etudiants/spelerin797/Documents/RI_TP1/classic_ir/IRSystem.py:35: SyntaxWarning: invalid escape sequence '\d'
  title_pattern = re.compile('(.*) \d+\.txt')


Reading in documents...
Stemming Documents...
A Winter Pilgrimage (1901) 0600121.txt
    Doc 1 of 60: A Winter Pilgrimage (1901)
A Yellow God an Idol of Africa 2857.txt
    Doc 2 of 60: A Yellow God an Idol of Africa
Allan Quatermain 711.txt
    Doc 3 of 60: Allan Quatermain
Allan and the Holy Flower 5174.txt
    Doc 4 of 60: Allan and the Holy Flower
Allan and the Ice Gods (1927) 0200201.txt
    Doc 5 of 60: Allan and the Ice Gods (1927)
Allan's Wife 2727.txt
    Doc 6 of 60: Allan's Wife
Ayesha, the Return of She 5228.txt
    Doc 7 of 60: Ayesha, the Return of She
Beatrice 3096.txt
    Doc 8 of 60: Beatrice
Benita, an African romance 2761.txt
    Doc 9 of 60: Benita, an African romance
Black Heart and White Heart 2842.txt
    Doc 10 of 60: Black Heart and White Heart
Cetywayo and his White Neighbours 0.txt
    Doc 11 of 60: Cetywayo and his White Neighbours
Child of Storm 1711.txt
    Doc 12 of 60: Child of Storm
Cleopatra 2769.txt
    Doc 13 of 60: Cleopatra
Colonel Quaritch, V.C. A

### Exercice 1. - Construction de l'index inversé

Ce premier exercice a pour objectif de construire l'index inversé non positionnel. L'attribut `self.inverted_index` est un tableau associatif qui associe une liste d'entiers (docIDs) à un mot (word).
 
Documentation ici https://docs.python.org/3/library/collections.html#collections.defaultdict. 

Exercice : modifier la fonction `index` pour calculer l'index inversé. 

Le résultat ci-dessous indique que vous avez réussi.
```
===== Running tests =====
Inverted Index Test
    Score: 3 Feedback: 5/5 Correct. Accuracy: 1.000000
```

In [13]:
print(dir(ir_system))

['_IRSystem__read_raw_data', '_IRSystem__read_stemmed_data', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'alphanum', 'boolean_retrieve', 'compute_tfidf', 'docs', 'get_posting', 'get_posting_unstemmed', 'get_tfidf', 'get_tfidf_unstemmed', 'get_uniq_words', 'index', 'inverted_index', 'p', 'process_query', 'query_rank', 'query_retrieve', 'rank_retrieve', 'read_data', 'tf', 'titles', 'vocab']


In [42]:
# Exercice 1. Indexation

def index(self):
	"""
	Construit l'index inversé et le stocke dans self.inverted_index.

	Dans le code ci-dessous, seul le dictionnaire des tokens est construit. Les postings lists sont vides.
	"""
	
	print("Indexing...")
	self.tf = defaultdict(Counter) # tf est un dictionnaire qui à un document associe un Counter de mots.
	inverted_index = defaultdict(list) # inverted_index est un dictionnaire qui à un mot associe une liste de documents. 

	for doc_id, doc in enumerate(self.docs):
		for w in doc :
			self.tf[doc_id][w] += 1

			if doc_id not in inverted_index[w]:
				inverted_index[w].append(doc_id)

	self.inverted_index = inverted_index

# Ne pas modifier les lignes ci-dessous
IRSystem.index = index
ir_system.index()
run_tests(ir_system, part=0)

Indexing...
===== Running tests =====
Inverted Index Test
    Score: 3 Feedback: 5/5 Correct. Accuracy: 1.000000


### Exercice 2. - Recherche booléenne

Ce deuxième exercice a pour objectif d'implémenter la recherche booléenne. La requête `query` est une liste de mots _lemmatisés_ et l'algorithme doit rechercher les documents qui contiennent TOUS ces mots. 


Exercice : modifier la fonction `boolean_retrieve` pour implémenter la recherche booléenne.


Le résultat ci-dessous indique que vous avez réussi.
```
===== Running tests =====
Boolean Retrieval Test
    Score: 3 Feedback: 5/5 Correct. Accuracy: 1.000000
```

In [62]:
# Exercice 2. Recherche booléenne
def boolean_retrieve(self, query):
	"""
	A partir d'une requête sous la forme d'une liste de mots *lemmatisés*,
	retourne la liste des documents dans lesquels *tous* ces mots apparaissent (ie une requête AND).
	Retourne une liste vide si la requête ne retourne aucun document.

	Dans le code ci-dessous, tous les documents répondent.
	"""
	
	docs = [i for i in range(len(self.docs))]
	for w in query :
		docs = list(set(docs) & set(self.get_posting(w)))
	return docs

# Ne pas modifier les lignes ci-dessous
IRSystem.boolean_retrieve = boolean_retrieve
run_tests(ir_system, part=1)

===== Running tests =====
Boolean Retrieval Test
    Score: 3 Feedback: 5/5 Correct. Accuracy: 1.000000


### Exercice 3. - Calcul des poids TF-IDF des termes dans les documents

Dans ce troisième exercice, l'objectif est de pré-calculer les poids TF-IDF pour chaque terme dans chaque document. Pour ce faire, appliquer la formule vue en cours. Utiliser le logarithme en base 10.


Exercice : modifier la fonction `boolean_retrieve` pour implémenter la recherche booléenne.

Le résultat ci-dessous indique que vous avez réussi.
```
Calculating tf-idf...
===== Running tests =====
TF-IDF Test
    Score: 3 Feedback: 5/5 Correct. Accuracy: 1.000000
```

In [71]:
# Exercice 3. calcul des scores tf-idf
from math import log10

def compute_tfidf(self):
	"""
	Calcule les scores tf-idf pour tous les mots de tous les documents et les stocke dans self.tfidf.

	Dans le code ci-dessous, les scores tf-idf sont tous nuls.
	"""
	print("Calculating tf-idf...")
	
	self.tfidf = defaultdict(Counter)
	N = len(self.docs)  # N = nombre de documents

	for word in self.vocab:
		for i in range(N):
			tf = self.tf[i][word]
			try:
				if(tf > 0) :
					df = len(self.get_posting(word))
					self.tfidf[i][word] = (1+log10(tf)) * log10(N/df)
				else :
					self.tfidf[i][word] = 0.
			except e:
				print(f"Erreur {e}")

# Ne pas modifier les lignes ci-dessous
IRSystem.compute_tfidf = compute_tfidf
ir_system.compute_tfidf()
run_tests(ir_system, part=2)

Calculating tf-idf...
===== Running tests =====
TF-IDF Test
    Score: 3 Feedback: 5/5 Correct. Accuracy: 1.000000


### Exercice 4. - Calcul de la similarité cosinus.

Dans ce troisième exercice, l'objectif est de calculer la similarité cosinus entre le vecteur requête `query` et chaque vecteur document. Pour ce faire, appliquer la formule vue en cours en considérant les éléments suivants :
- Ne considérer que la mesure TF pour calculer le poids des termes de la requête (on n'utilise pas IDF).
- Utiliser le logarithme en base 10. 

Exercice : modifier la fonction `rank_retrieve` pour implémenter la recherche booléenne.

Le résultat ci-dessous indique que vous avez réussi.
```
===== Running tests =====
Cosine Similarity Test
    Score: 3 Feedback: 5/5 Correct. Accuracy: 1.000000
```

In [130]:
# Exercice 4. Similarité cosinus
from math import log10, sqrt

def rank_retrieve(self, query):
	"""
	A partir d'une requête (une liste de mots), retourne une liste de documents (classés par docID) et de scores pour la requête en appliquant la simalirité cosinus.

	Dans l'exemple ci-dessous. C'est la mesure de Jaccard qui est utilisée.
	"""
	scores = [0.0 for _ in range(len(self.docs))]

	query_set = set(query)
	
	query_tfidf = {}
	for w in query_set :
		query_tfidf[w] = 1+log10(query.count(w))
	
	for doc_id, doc in enumerate(self.docs):
		qd = sum(query_tfidf[w] * self.tfidf[doc_id][w] for w in query_set)
		d2 = sum(v * v for v in self.tfidf[doc_id].values())
			
		scores[doc_id] = qd / sqrt(d2) if d2 > 0 else 0.0
		
	# Tri des scores
	ranking = [idx for idx, sim in sorted(enumerate(scores), key=lambda pair: pair[1], reverse=True)]
	results = []

	for i in range(10):
		results.append((ranking[i], scores[ranking[i]]))

	print(f"{results}\n")
	return results

# Ne pas modifier les lignes ci-dessous
IRSystem.rank_retrieve = rank_retrieve
run_tests(ir_system, part=3)

===== Running tests =====
Cosine Similarity Test
[(56, 0.010676611271744088), (25, 0.008463116269726562), (49, 0.008462987843582521), (37, 0.00839586658084902), (18, 0.008360056776471416), (54, 0.008320124278304216), (5, 0.008316020650549037), (41, 0.008312726995818018), (29, 0.00785209214713161), (44, 0.007814399700034913)]

[(59, 0.05041279333939136), (31, 0.042933912797404966), (16, 0.04245722537237054), (36, 0.0391023868431653), (3, 0.03250369701658961), (45, 0.03200276000633705), (40, 0.028353460953342218), (1, 0.02298825032652949), (6, 0.022750210842425178), (48, 0.022439179053556423)]

[(15, 0.03405789447552078), (41, 0.02218901367816242), (39, 0.021800770675665816), (13, 0.016669193743447684), (16, 0.010152619001197564), (52, 0.009352842834467906), (27, 0.00861473088344378), (50, 0.008208957726149083), (29, 0.008035449433972445), (5, 0.00746212911745865)]

[(24, 0.015298081869538089), (21, 0.014226522006871), (45, 0.01402257841496572), (56, 0.013840806806046085), (5, 0.01314118